In [ ]:
from google.colab import drive
import pandas as pd
from scipy.stats import norm
import scipy.stats as stats
import numpy as np
from scipy.fft import fft, fftfreq
import statistics
import scipy.integrate as integrate
import scipy.stats as stats
import matplotlib.pyplot as plt
import re
from scipy.integrate import trapz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import zipfile
import shutil
from scipy.stats import ttest_ind
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, f1_score, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import metrics
import pickle
import seaborn as sns
import heapq


#Read
Read the game dataset that was gathered from the mobile game and fix the missing places

In [ ]:
#In the recorded game datset there are empty spaces in the Type column, this function fills those empty cells
def fill_type(data):
  Value = data[0]['Type'].unique()
  for df in data:
    df['Type'] = df['Type'].replace(Value[1], method = 'ffill')
    df['Type'] = df['Type'].replace(Value[4], Value[5])

In [ ]:
#Extrac the 200 rows of the game dataset which belongs to the main phase which is the target for the analysis
def main_data(data):
  main = []
  Value = data[0]['Type'].unique()
  for df in data:
    main.append(df[df['Type'] == Value[-3]])
  return main

In [ ]:
#Create the main dataset
def extract_main_test(filepath):
  GAME = pd.read_excel(filepath, sheet_name = None)
  game_data = list(GAME.values())
  fill_type(game_data)
  main_game = main_data(game_data)
  return main_game

#IVA Features
Definition of the IVA-2 features. These functions are defined based on the standard definition of IVA-2 features.

In [ ]:
def idx():
  trial = 'AVAAVVVVAAVAAVAVVAVAAAVAVAVVVAVAAAVVAAVVAVAAAVAVVV'*8
  auditory_idx = np.array([i.start() for i in re.finditer('A', trial)])
  visual_idx = np.array([i.start() for i in re.finditer('V', trial)])
  return [auditory_idx, visual_idx]

In [ ]:
def Prudence(data):
  auditory_prudence = []
  visual_prudence = []
  error_idx = np.array([6, 12, 19, 26, 31, 38, 44, 49, 51, 106, 112, 119, 126, 131, 138, 144, 149, 151,
      206, 212, 219, 226, 231, 238, 244, 249, 251, 306, 312, 319, 326, 331, 338, 344, 349, 351,
      52, 53, 152, 153, 252, 253, 352, 353, 59, 61, 68, 73, 80, 85, 87, 91, 155, 159, 161,
      168, 173, 180, 185, 187, 191, 255, 259, 261, 268, 273, 280, 285, 287, 291,
      355, 359, 361, 368, 373, 380, 385, 387, 3])-1
  [auditory_idx, visual_idx] = idx()
  aud_error = np.intersect1d(auditory_idx, error_idx)
  vis_error = np.intersect1d(visual_idx, error_idx)
  for df in data:
    aud_data = df.iloc[list(aud_error)]
    aud = aud_data[aud_data['Response'] == '  comission error  '].shape[0]
    auditory_prudence.append(100-((aud/44)*100))

    vis_data = df.iloc[list(vis_error)]
    vis = vis_data[vis_data['Response'] == '  comission error  '].shape[0]
    visual_prudence.append(100-((vis/35)*100))
  return [auditory_prudence, visual_prudence]

In [ ]:
def Vigilance(data):
  auditory_vigilance = []
  visual_vigilance = []
  error_idx = np.array([56, 62, 69, 76, 81, 88, 94, 99, 101, 102,
              156, 162, 169, 176, 181, 188, 194, 199, 201, 202,
              256, 262, 269, 276, 281, 288, 294, 299, 301, 302,
              356, 362, 369, 376, 381, 388, 394, 399, 13, 20, 27, 32, 39, 45, 50,
              107, 113, 120, 127, 132, 139, 145, 150,
              207, 213, 220, 227, 232, 239, 245, 250,
              307, 313, 320, 327, 332, 339, 345, 350])-1
  [auditory_idx, visual_idx] = idx()
  aud_error = np.intersect1d(auditory_idx, error_idx)
  vis_error = np.intersect1d(visual_idx, error_idx)
  for df in data:
    aud_data = df.iloc[list(aud_error)]
    aud = aud_data[aud_data['Response'] == '  omission error  '].shape[0]
    auditory_vigilance.append(100-((aud/35)*100))

    vis_data = df.iloc[list(vis_error)]
    vis = vis_data[vis_data['Response'] == '  omission error  '].shape[0]
    visual_vigilance.append(100-((vis/34)*100))




  return [auditory_vigilance, visual_vigilance]

In [ ]:
def Comprehension(data):
  auditory_comp = []
  visual_comp = []
  error_idx = np.array([54, 55, 57, 58, 60, 63, 64, 65, 66, 67, 70, 71, 72, 74,
                        75, 77, 78, 79, 82, 83, 84, 86, 89, 90, 92, 93, 95, 96, 97, 98, 100,
                        154, 157, 158, 160, 163, 164, 165, 166, 167, 170, 171, 172, 174, 175,
                        177, 178, 179, 182, 183, 184, 186, 189, 190, 192, 193, 195, 196, 197, 198, 200,
                        254, 257, 258, 260, 263, 264, 265, 266, 267, 270, 271, 272, 274, 275, 277, 278, 279, 282,
                        283, 284, 286, 289, 290, 292, 293, 295, 296, 297, 298, 300, 354, 357, 358, 360, 363, 364,
                        365, 366, 367, 370, 371, 372, 374, 375, 377, 378, 379, 382, 383, 384, 386, 389, 390, 392, 393, 395,
                        396, 397, 398, 400, 1, 2, 3, 4, 5, 8, 9, 10, 11, 14, 15, 16, 17, 18, 21, 22, 23, 24, 25,
                        28, 29, 30, 33, 34, 35, 36, 37, 40, 41, 42, 43, 46, 47, 48, 103, 104, 105, 108, 109, 110,
                        111, 114, 115, 116, 117, 118, 121, 122, 123, 124, 125, 128, 129, 130, 133, 134, 135, 136,
                        137, 140, 141, 142, 143, 146, 147, 148, 203, 204, 205, 208, 209, 210, 211, 214, 215, 216, 217,
                        218, 221, 222, 223, 224, 225, 228, 229, 230, 233, 234, 235, 236, 237, 240, 241, 242, 243, 246, 247, 248,
                        303, 304, 305, 308, 309, 310, 311, 314, 315, 316, 317, 318, 321, 322, 323, 324, 325, 328,
                        329, 330, 333, 334, 335, 336, 337, 340, 341, 342, 343, 346, 347, 348])-1
  [auditory_idx, visual_idx] = idx()
  aud_error = np.intersect1d(auditory_idx, error_idx)
  vis_error = np.intersect1d(visual_idx, error_idx)
  for df in data:
    aud_data = df.iloc[list(aud_error)]
    aud = aud_data[(aud_data['Response'] == '  omission error  ') | (aud_data['Response'] == '  comission error  ')].shape[0]
    auditory_comp.append(100-((aud/121)*100))

    vis_data = df.iloc[list(vis_error)]
    vis = vis_data[(vis_data['Response'] == '  omission error  ') | (vis_data['Response'] == '  comission error  ')].shape[0]
    visual_comp.append(100-((vis/130)*100))



  return [auditory_comp, visual_comp]

In [ ]:
def Consistency(data):
  consistencyA = []
  consistencyV = []
  for df in data:

    reaction_timeA = list(df[((df['Response'] == '  Hit  ') | (df['Response'] == '  comission error  ')) &
                 ((df['Mode'] == '  Fish Auditory  ') | (df['Mode'] == '  Shark Auditory  '))]['ReactionTime'])
    q1_A = np.percentile(reaction_timeA, 25)
    q3_A = np.percentile(reaction_timeA, 75)
    consistencyA.append((q1_A/q3_A)*100)

    reaction_timeV = list(df[((df['Response'] == '  Hit  ') | (df['Response'] == '  comission error  ')) &
                 ((df['Mode'] == '  Fish Visual  ') | (df['Mode'] == '  Shark Visual  '))]['ReactionTime'])
    q1_V = np.percentile(reaction_timeV, 25)
    q3_V = np.percentile(reaction_timeV, 75)
    consistencyV.append((q1_V/q3_V)*100)

  return [consistencyA, consistencyV]

In [ ]:
def Stamina(data):
  StaminaA = []
  StaminaV = []
  for df in data:
    df1 = df.head(n = 200)
    Auditory_mean1 = statistics.mean(list(df1[(df1['Response'] == '  Hit  ') &
             ((df1['Mode'] == '  Fish Auditory  ') | (df1['Mode'] == '  Shark Auditory  '))]['ReactionTime']))

    Visual_mean1 = statistics.mean(list(df1[(df1['Response'] == '  Hit  ') &
                 ((df1['Mode'] == '  Fish Visual  ') | (df1['Mode'] == '  Shark Visual  '))]['ReactionTime']))

    df2 = df.tail(n = 200)
    Auditory_mean2 = statistics.mean(list(df2[(df2['Response'] == '  Hit  ') &
                 ((df2['Mode'] == '  Fish Auditory  ') | (df2['Mode'] == '  Shark Auditory  '))]['ReactionTime']))


    Visual_mean2 = statistics.mean(list(df2[(df2['Response'] == '  Hit  ') &
                 ((df2['Mode'] == '  Fish Visual  ') | (df2['Mode'] == '  Shark Visual  '))]['ReactionTime']))

    StaminaA.append((Auditory_mean1/Auditory_mean2)*100)
    StaminaV.append((Visual_mean1/Visual_mean2)*100)

  return [StaminaA, StaminaV]

In [ ]:
def Focus(data):

  FocusA = []
  FocusV = []

  for df in data:
    reaction_timeA = list(df[((df['Response'] == '  Hit  ') | (df['Response'] == '  comission error  ')) &
                 ((df['Mode'] == '  Fish Auditory  ') | (df['Mode'] == '  Shark Auditory  '))]['ReactionTime'])
    FocusA.append((1 - (statistics.pstdev(reaction_timeA)/statistics.mean(reaction_timeA)))*100)

    reaction_timeV = list(df[((df['Response'] == '  Hit  ') | (df['Response'] == '  comission error  ')) &
                 ((df['Mode'] == '  Fish Visual  ') | (df['Mode'] == '  Shark Visual  '))]['ReactionTime'])
    FocusV.append((1 - (statistics.pstdev(reaction_timeV)/statistics.mean(reaction_timeV)))*100)

  return [FocusA, FocusV]

In [ ]:
def Speed(data):
  SpeedA = []
  SpeedV = []
  for df in data:
      reaction_timeA = list(df[((df['Response'] == '  Hit  ') &
                 ((df['Mode'] == '  Fish Auditory  ') | (df['Mode'] == '  Shark Auditory  ')) &
                  (df['middleMiddle'] == 1) & (df['ReactionTime'] > 0.125))]['ReactionTime'])
      SpeedA.append(statistics.mean(reaction_timeA))

      reaction_timeV = list(df[((df['Response'] == '  Hit  ') &
                 ((df['Mode'] == '  Fish Visual  ') | (df['Mode'] == '  Shark Visual  ')) &
                  (df['middleMiddle'] == 1) & (df['ReactionTime'] > 0.125))]['ReactionTime'])

      SpeedV.append(statistics.mean(reaction_timeV))

  return [SpeedA, SpeedV]

#Create Dataset
Functions that are used for creating the feature set

In [ ]:
#Calculating the selected features
def extract_features(data):
  features = Consistency(data) + Stamina(data) + [Vigilance(data)[0]] + [Comprehension(data)[1]]
  columns = ['Auditory Consistancy', 'Visual Consistancy', 'Auditory Stamina', 'Visual stamina', 'Auditory Vigilance', 'Visual Comprehension']
  data_dict = dict(zip(columns, features))
  dataset = pd.DataFrame.from_dict(data_dict)
  return dataset

#IVA Set

In [ ]:
#If the datset is for one person use this normalization function
def normal_one(data):
  min_values = [22.87627949072171, 20.65247385818543, 84.20707555042883, 85.38833884113552, 57.14285714285715, 51.53846153846154]
  max_values = [86.88650721733065, 85.41685682241307, 150.9936660303136, 139.1899024268303, 100.0, 100.0]
  norm_data = (data - min_values)/list(map(lambda a, b: a - b, max_values, min_values))
  return norm_data

In [ ]:
#If the dataset is for multiple people use this normalization function
def normalize(df):
  scaler = MinMaxScaler()
  cols = list(df.columns)
  data = df[cols]
  d = scaler.fit_transform(data)
  normal_df = pd.DataFrame(d, columns=cols)
  return normal_df

In [ ]:
#Replace filepath with the path which the game dataset is stored
filepath = '...'

df = extract_features(extract_main_test(filepath))
#normalize the data using either of those two normalization functions
df_norm = normalization(df)

#Label Result

In [ ]:
#load the model
#replace the model_filepath with the path the model is stored
game_model1 = pickle.load(open("model_filepath", 'rb'))
y_pred = game_model1.predict(df_norm)